In [1]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

# Read in Data

In [2]:
%run "Data Collection and Cleaning.ipynb"

C:\Users\davis\miniconda3\lib\site-packages\scipy\stats\stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [3]:
#create dataframe with the high correlation columns, and split up into train and test data
high_corr_df = df_merged_train_data.copy()
high_corr_df = high_corr_df[high_corr_rank_cols]

train, test = high_corr_df[0:0], high_corr_df[0:0] 
for team in high_corr_df.abbreviation.unique():
    
    r_season = int(np.random.randint(2017, 2020))
#     print(team, r_season)
    
    temp_train = high_corr_df[(high_corr_df.Season != r_season) & (high_corr_df.abbreviation == team)]
    temp_test = high_corr_df[(high_corr_df.Season == r_season) & (high_corr_df.abbreviation == team)]
    
    train = train.append(temp_train)
    test = test.append(temp_test)
    
train = train.sample(train.shape[0])
test = test.sample(test.shape[0])

x_train = train[high_corr_rank_cols].set_index(['abbreviation', 'Season']).drop("rank", axis = 1)
y_train = train["rank"].values.reshape(-1, 1)

x_test = test[high_corr_rank_cols].set_index(['abbreviation', 'Season']).drop("rank", axis = 1)
y_test = test["rank"].values.reshape(-1, 1)

# y_data = high_corr_df["rank"].values.reshape(-1, 1)
# x_data = high_corr_df[high_corr_rank_cols].set_index(['abbreviation', 'Season']).drop('rank', axis = 1)


# x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, random_state = 42)

In [4]:
#Train Linear Rregression Model
rank_regr = LinearRegression().fit(x_train, y_train)

y_predict_train = rank_regr.predict(x_train)
y_predict_test = rank_regr.predict(x_test)

Since sklearn accuracy function is based on exact matches, it is tough to use the accuracy_score function since it will compare a decimal to an integer. So we will test in two different ways. The first test will be by rounding to the nearest integer, and comparing that result using the accuracy score. The seconcd test will be to get the rate of wins within X of the true value, where X is user defined.

Test #1

In [5]:
def rounded_accuracy(y_actual, y_pred):
    round_y_predict = []

    for y in y_pred:
        rounded_y = round(float(y))
        round_y_predict.append(rounded_y)
        
    round_y_predict_arr = np.array(round_y_predict)
    accuracy_sc = accuracy_score(y_actual, round_y_predict_arr)

    return accuracy_sc

In [6]:
train_accuracy = rounded_accuracy(y_train,y_predict_train)
print("Training Accuracy based on rounded: ",train_accuracy)

test_accuracy = rounded_accuracy(y_test,y_predict_test)
print("Test Accuracy based on rounded: ",test_accuracy)

Training Accuracy based on rounded:  0.1
Test Accuracy based on rounded:  0.1


Test #2

In [7]:
def calc_accuracy_rate(match_bool):
    true_count = 0
    total_matches = len(match_bool)
    for match in match_bool:
        if match:
            true_count += 1

    return true_count / total_matches

In [8]:
#Read linear Regression Model's Accuracy
def get_accuracy(num_wins_window, y_predict, y_actual):
    # if number of wins predicted is within 5 of the actual number of wins 
    # then this will be marked as a "success"
    # (for example if actual number of wins is 65, "correct window" is 60-70)
    # this can easily be adjusted based on the users desired level of accuracy

    round_y_predict = []

    for i,y in enumerate(y_predict):
        wins_diff = abs(round(float(y)) - y_actual[i])
        if wins_diff <= num_wins_window:
            match = True
        else:
            match = False
        round_y_predict.append(match)

        
    # round_y_predict_arr = np.array(round_y_predict)
    train_accuracy = calc_accuracy_rate(round_y_predict)

    return train_accuracy

In [9]:
train_accuracy_0 = get_accuracy(0,y_predict_train,y_train)
print("Exact Accuracy:",train_accuracy_0)

train_accuracy_1 = get_accuracy(1,y_predict_train,y_train)
print("Rate of correct wins within 1:", train_accuracy_1)

Exact Accuracy: 0.1
Rate of correct wins within 1: 0.38333333333333336


In [10]:
test_accuracy_0 = get_accuracy(0,y_predict_test,y_test)
print("Exact Accuracy:",test_accuracy_0)

test_accuracy_1 = get_accuracy(1,y_predict_test,y_test)
print("Rate of correct wins within 1:", test_accuracy_1)


Exact Accuracy: 0.1
Rate of correct wins within 1: 0.2


With the High Correlation variables, all games are correctly predicted within one win. This is for both Test and Training datasets